In [ ]:
# TO DO:
    # convert "a","b" into numbers 1-8 (takes up less memory)
    # make sure only two cols ("rank" & "file")
    # create pivot table iteratively (like streaming)
        # so that the whole thing doesn't have to be held in memory
        # maybe has to be done manually
        
# https://lethain.com/handling-very-large-csv-and-xml-files-in-python/
# https://stackabuse.com/read-a-file-line-by-line-in-python/
        

In [1]:
print("Is this updated to GIT?")

Is this updated to GIT?


In [1]:
import os
import sys
import pandas as pd
import chess.pgn as pychess
from chess import square_name
from chess import SQUARE_NAMES, BB_SQUARES
import seaborn as sns
import ChessFuncs
import bz2
import time
import io

def unzip(zipped_pgn):
    '''
    unzips .bz2 pgn files: zipped_pgn
    maybe won't work if "data" variable is saved to RAM, as some unzipped files will be 20GB+ !
    '''
    
    filepath = zipped_pgn
    zipfile = bz2.BZ2File(filepath)
    data = zipfile.read()
    newfilepath = "pgn1.txt"
    open(newfilepath, 'wb').write(data) 

def parse_pgn(pgn_list):
    '''
    1) takes a list of pgn lines
    2) writes line to file on disk 
    3) returns a py-chess pgn object
    '''
    
    with open("pgn-chunk.txt", "w") as pgn_chunk:
        for line in pgn_list:
            pgn_chunk.write(line)
            
    parsed_pgn = pychess.read_game(open("pgn-chunk.txt"))
    return parsed_pgn

def num_chess_games(pgn_path):
    '''
    Uses the decompressed lichess.org pgn file sizes to predict the number of chess games inside.
    Means a for loop doesn't have to load the entire thing before hand.
    '''
    pgn_size = os.stat(pgn_path).st_size
    x = 121332/92811021
    return round(pgn_size*x)

def cpuTime(t="days"):
    '''Estimated of the total time to process 400 million games of chess'''
    
    mins = (1300/450) * 7
    x = (409700297/121332)
    hours = (mins * x)/60
    days = hours/24
    
    if t == "hours":
        return round(hours)
    if t == "days":
        return round(days)

In [2]:
in_path = "/Volumes/SAMSUNG EVO SSD 500GB/pgn3.txt"
temp_path = "/Volumes/SAMSUNG EVO SSD 500GB/pgn-chunk.txt"
out_path = "/Volumes/SAMSUNG EVO SSD 500GB/chess-results-3.txt"

In [3]:
# for file...pgn.bz2 in os.listdir(directory):
#     file = unzip(file)
#     in_path = file.name[:-4]
#     ...
#     os.remove(in_path)
    
print("Started:",time.asctime())
with open(in_path) as bigfile:
    
    count_lines = 0
    count_games = 0
    pgn = []
    while count_games < num_chess_games(in_path): # each loop this moves on 1x line
        
        try:

            line = bigfile.readline()
            count_lines += 1

            if line.startswith("[Event"):
                pgn.append(line) # appends the FIRST line of the pgn

            elif line.startswith("1. "):
                pgn.append(line) # appends the FINAL line of the pgn
                pgn = "".join(pgn) # concatenate pgn list into string
                pgn = io.StringIO(pgn) # create virtual pgn file in RAM using StringIO
                pgn = pychess.read_game(pgn) # create virtual pgn py-chess object
                end_board = ChessFuncs.end_board(pgn)

                king_num = ChessFuncs.get_check_mate_pos_3(pgn) #!!!! NEED to make sure loser is only 1 or 0, not "*"
                with open(out_path, "a") as results: #!!!! NEED to save name marker in case of crash
                    if end_board.is_checkmate() and (pgn.headers["Result"] in ["1-0","0-1"]):
                        results.write(f"{king_num}\n") # writes results
                    else:
                        pass

                pgn = [] # resets pgn for next game loop
                count_games += 1

            else:
                pgn.append(line) # appends all INBETWEEN lines of the PGN
        
        except Exception as e:
            pass
            
print("Finished:",time.asctime())


Started: Sat Aug 18 00:15:27 2018


KeyboardInterrupt: 

In [9]:
count_games

374666

In [ ]:
#gets position of queen if and only if queen is checkmating
pgn = pychess.read_game(open("pgn-chunk.txt"))
board = pgn.board()
board = ChessFuncs.end_board(pgn)
print(str(board.attackers(white,24))) # needs to know if queen is attacking king square
print("\n")
print(str(board))
board

In [ ]:
#gets position of queen, check agains 'sq'
sq = [i for i in zip(SQUARE_NAMES,BB_SQUARES)]

queen = 5
white = 1
black = 0
x = board.pieces(queen, black)
x.mask